In [1]:
from fastapi import FastAPI
from gdeltdoc import GdeltDoc, Filters
import pandas as pd
import os
import datetime

In [2]:
# PARA LA FUNCION DE LIMPIEZA, FUNCION AUXILIAR
def limpiar_dataframe(df: pd.DataFrame):
    # Nos aseguramos que el DataFrame tenga todas las fechas. Si alguna no está, rellenamos con 0.
    start_date = df['datetime'].min()
    end_date = df['datetime'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    df = df.set_index('datetime').reindex(all_dates).fillna(0).reset_index()
    df = df.rename(columns={'index': 'datetime'}) # Renombramos la columna del índice
    df_limpio= df.dropna().drop_duplicates()
    return df_limpio # creo la funcion primero, que luego podre usar para cualquier DataFrame. 
                    #Metemos solo los nulos y duplicados en un primer momento

In [3]:
def extraccion_total(funcion_obtener_datos: str, keyword: str, time: str):
    countries = ['AU', 'BE', 'SZ', 'CY', 'EZ', 'GM', 'DA', 'GR', 'SP', 'FI', 'FR', 'EI', 'IT', 'LU', 'NL', 'NO', 'PO', 'SW', 'SI']
    
    for country in countries:
        f = Filters(keyword=keyword,
                    start_date="2017-01-01",
                    end_date="2024-04-05",
                    country=country)
        gd = GdeltDoc() 
        
        if funcion_obtener_datos.lower() == "tone":
            results = gd.timeline_search("timelinetone", f)
            df = pd.DataFrame(results)
            folder_name = "tono"
        elif funcion_obtener_datos.lower() == "popularity":
            results = gd.timeline_search("timelinevol", f)
            df = pd.DataFrame(results)
            folder_name = "popularity"
        else:
            return "Error: La función de extracción de datos especificada no es válida."
        
        if time.lower() == "mensual":
            df['datetime'] = pd.to_datetime(df['datetime'])
            df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()
            save_folder = f"./data_final/{folder_name}/mensual"
            df_save = df_monthly
        elif time.lower() == "trimestral":
            df['datetime'] = pd.to_datetime(df['datetime'])
            df_trimestral = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index() 
            save_folder = f"./data_final/{folder_name}/trimestral"
            df_save = df_trimestral
            
        else:
            return "Error: El intervalo especificado no es válido."
        
        df_save = limpiar_dataframe(df_save)
        
        # Verificar y crear la carpeta si no existe
        os.makedirs(save_folder, exist_ok=True)

        # y guardamos
        file_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S") # para evitar sobreescribir
        filename = f"{save_folder}/csv_{folder_name}_{time}_{country}_{keyword}_{file_timestamp}.csv"  # Agregar el nombre del país y el periodo de tiempo al archivo CSV
        df_save.to_csv(filename, index=False)
    
    return f"Media de {funcion_obtener_datos.capitalize()} data downloaded successfully for all countries"

# ULTIMISIMO CAMBIO, CREAMOS DOS LLAMADAS NUEVAS, UNA PARA DATOS MENSUALES Y OTRA PARA TRIMESTRALES DE TONO Y POPULARIDAD
# creamos una ultima llamada que aune todo, para la extraccion completa por países de tono y popularidad
# 

In [4]:
# Hacemos pruebas con cada ejemplo
extraccion_total("tone","Inclusive growth", "mensual")

'Media de Tone data downloaded successfully for all countries'

In [5]:
extraccion_total("tone","Inclusive growth", "trimestral")

'Media de Tone data downloaded successfully for all countries'

In [6]:
extraccion_total("popularity","Inclusive growth", "mensual")

'Media de Popularity data downloaded successfully for all countries'

In [7]:
extraccion_total("popularity","Inclusive growth", "trimestral")

'Media de Popularity data downloaded successfully for all countries'

In [8]:
# otra prueba, para COMPROBAR SI LA FUNCION DE LIMPIEZA FUNCIONA MENSUAL Y TRIMESTRAL
# PARA LA FUNCION DE LIMPIEZA, FUNCION AUXILIAR
def limpiar_dataframe(df: pd.DataFrame,freq:str): # PARA EL TIPO DE FRECUENCIA QUE QUEREMOS
    # Nos aseguramos que el DataFrame tenga todas las fechas. Si alguna no está, rellenamos con 0.
    start_date = df['datetime'].min()
    end_date = df['datetime'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq=freq)
    df = df.set_index('datetime').reindex(all_dates).fillna(0).reset_index()
    df = df.rename(columns={'index': 'datetime'}) # Renombramos la columna del índice
    df_limpio= df.dropna().drop_duplicates()
    return df_limpio # creo la funcion primero, que luego podre usar para cualquier DataFrame. 
                    #Metemos solo los nulos y duplicados en un primer momento

In [9]:
def extraccion_total(funcion_obtener_datos: str, keyword: str, time: str):
    countries = ['AU', 'BE', 'SZ', 'CY', 'EZ', 'GM', 'DA', 'GR', 'SP', 'FI', 'FR', 'EI', 'IT', 'LU', 'NL', 'NO', 'PO', 'SW', 'SI']
    
    for country in countries:
        f = Filters(keyword=keyword,
                    start_date="2017-01-01",
                    end_date="2024-04-05",
                    country=country)
        gd = GdeltDoc() 
        
        if funcion_obtener_datos.lower() == "tone":
            results = gd.timeline_search("timelinetone", f)
            df = pd.DataFrame(results)
            folder_name = "tono"
        elif funcion_obtener_datos.lower() == "popularity":
            results = gd.timeline_search("timelinevol", f)
            df = pd.DataFrame(results)
            folder_name = "popularity"
        else:
            return "Error: La función de extracción de datos especificada no es válida."
        
        if time.lower() == "mensual":
            df['datetime'] = pd.to_datetime(df['datetime'])
            df_monthly = df.groupby(pd.Grouper(key='datetime', freq='M')).sum().reset_index()
            save_folder = f"./data_final/{folder_name}/mensual"
            df_save = df_monthly
            df_save = limpiar_dataframe(df_save, "M")
        elif time.lower() == "trimestral":
            df['datetime'] = pd.to_datetime(df['datetime'])
            df_trimestral = df.groupby(pd.Grouper(key='datetime', freq='Q')).sum().reset_index() 
            save_folder = f"./data_final/{folder_name}/trimestral"
            df_save = df_trimestral
            df_save = limpiar_dataframe(df_save,"Q")
            
        else:
            return "Error: El intervalo especificado no es válido."
        

        os.makedirs(save_folder, exist_ok=True) # para asegurarnos de que la carpeta este

        # y guardamos
        file_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S") # para evitar sobreescribir
        filename = f"{save_folder}/{keyword}_csv_{folder_name}_{time}_{country}_{file_timestamp}.csv"  # Agregar el nombre del país y el periodo de tiempo al archivo CSV
        df_save.to_csv(filename, index=False)
    
    return f"Mean of {funcion_obtener_datos.capitalize()} data downloaded successfully for all countries"


In [10]:
# Hacemos pruebas con cada ejemplo
extraccion_total("tone","Inclusive growth", "mensual")
# Y FUNCIONA

'Mean of Tone data downloaded successfully for all countries'

In [11]:
extraccion_total("tone","Inclusive growth", "trimestral")

'Mean of Tone data downloaded successfully for all countries'